# Composing techniques: Digital Dynamical Decoupling and Zero Noise Extrapolation

Noise in quantum computers can arise from a variety of sources, and sometimes applying multiple error mitigation techniques can be more beneficial than applying a single technique alone. 

Here we apply a combination of Digital Dynamical Decoupling (DDD) and Zero Noise Extrapolation (ZNE) on a GHZ state.

In [DDD](../guide/ddd.md), the input quantum circuit is modified by adding in gate sequences at regular intervals designed to reduce interaction between (i.e., decouple) the qubits from their environment. 

In [ZNE](../guide/zne.md), the expectation value of the observable of interest is computed at different noise levels, and subsequently the ideal expectation value is inferred by extrapolating the measured results to the zero-noise
limit. 
More information on the DDD and ZNE techniques can be found in the corresponding sections of the user guide (linked
above).

## Setup

We begin by importing the relevant modules and libraries required for the rest of this tutorial.

In [ ]:
import cirq
import numpy as np
from mitiq import MeasurementResult, Observable, PauliString

## Task

We will demonstrate using DDD + ZNE on on a GHZ state.

In [ ]:
# TODO: Does this circuit include measurements by default?
def ghz(num_qubits):
    # Create  qubit registers
    qubits = cirq.LineQubit.range(num_qubits)

    # Create a quantum circuit
    circuit = cirq.Circuit()
    # Add a Hadamard gate to the first qubit
    circuit.append(cirq.H(qubits[0]))

    # Add CNOT gates to entangle the first qubit with each of the other qubits
    for i in range(1, num_qubits):
        circuit.append(cirq.CNOT(qubits[0], qubits[i]))

    return circuit

In [ ]:
num_qubits = 6
circuit = ghz(num_qubits)
print(circuit)

## Noise model and executor

**Importantly**, since DDD is designed to mitigate time-correlated (non-Markovian) noise, we simulate systematic $R_z$ rotations and depolarising noise applied to each qubit after each time step. This corresponds to noise which is strongly time-correlated and, therefore, likely to be mitigated by DDD.

We use an [executor function](../guide/executors.md) to run the quantum circuit with the noise model applied.

In [ ]:
# TODO: does this executor include measurements? No. What does `with_noise` do?
def execute(
    circuit: cirq.Circuit, 
    rz_noise: float = 0.01,
    depolar_noise: float = 0.0005
    ) -> MeasurementResult:
    """
    Execute a circuit with R_z dephasing noise of strength ``rz_noise`` and 
    depolarizing noise ``depolar_noise``
    """
    # Simulate systematic dephasing (coherent RZ) on each qubit for each moment.
    #measurements = circuit[-1]

    #circuit = circuit[:-1]
    circuit = circuit.with_noise(cirq.rz(rz_noise))

    # Simulate systematic depolarizing on each qubit for each moment.
    circuit = circuit.with_noise(cirq.bit_flip(depolar_noise))

    circuit += cirq.measure(*sorted(circuit.all_qubits()), key="m")

    #circuit.append(measurements)
    simulator = cirq.DensityMatrixSimulator()

    result = simulator.run(circuit, repetitions=1000)
    # print(result.measurements["m"])
    bitstrings = result.measurements["m"]
    # print(bitstrings)
    return MeasurementResult(bitstrings)

In [ ]:
execute(circuit)

## Observable

In this example, we just want to check if we have achieved entanglement across all qubits. In this case, we will measure the observable $⨂_{i=1}^n​X_i$ or measuring `X` on all qubits. 

This corresponds to projecting the state onto either the $∣+⟩^{⊗n}$ or $∣−⟩^{⊗n}$ basis. For a perfect GHZ state, this observable will have an expectation value of 1, which corresponds to all qubits being in the same state.

In [ ]:
obs = Observable(PauliString("X" * num_qubits))
print(obs)

For the circuit defined above, the ideal (noiseless) expectation value of the observable is 1, as we will see though, the unmitigated (noisy) result is impacted by depolarizing and readout errors.

In [ ]:
from functools import partial

ideal_exec = partial(execute, rz_noise = 0.0, depolar_noise = 0.0)

ideal = obs.expectation(circuit, ideal_exec)
print("Ideal value:", "{:.5f}".format(ideal.real))

In [ ]:
noisy_exec = execute #partial(execute, rz_noise = 0.05, depolar_noise = 0.00)
noisy = obs.expectation(circuit, noisy_exec) # TO-DO is this an in-place operation?
print("Unmitigated noisy value:", "{:.5f}".format(noisy.real))

Next we choose our gate sequences to be used in the digital dynamical decoupling routine (DDD). 
More information on choosing appropriate sequences can be found in the [DDD theory](../guide/ddd-5-theory.md#common-examples-of-ddd-sequences) section of the user guide.

In [ ]:
from mitiq import ddd

rule = ddd.rules.xyxy

# TODO: Try deeper circuit? Add artificially long idle window. 
ddd_executor = ddd.mitigate_executor(noisy_exec, observable=obs, rule=rule)

ddd_result = ddd_executor(circuit)
print("Mitigated value obtained with DDD:", "{:.5f}".format(ddd_result.real))

For comparison, we then apply ZNE without DDD.

In [ ]:
from mitiq import zne

zne_executor = zne.mitigate_executor(noisy_exec, observable=obs, scale_noise=zne.scaling.folding.fold_global)
zne_result = zne_executor(circuit)
print("Mitigated value obtained with ZNE:", "{:.5f}".format(zne_result.real))

Finally, we apply a combination of DDD and ZNE.
DDD is applied first to apply the control pulses to each circuit which ZNE runs to do its extrapolation.

In [ ]:
combined_executor = zne.mitigate_executor(ddd_executor, observable=obs, scale_noise=zne.scaling.folding.fold_global)

combined_result = combined_executor(circuit)
print("Mitigated value obtained with DDD + ZNE:", "{:.5f}".format(combined_result.real))

From this example we can see that each technique affords some improvement, and for this specific noise model, the combination of DDD and ZNE is more effective in mitigating errors than either technique alone.